[DIY Covid-19 Region Dashboard](https://github.com/fsmeraldi/diy-covid19dash) 

# DIY Covid-19 Region Dashboard

DIY Covid-19 Dashboard displaying data by region in England. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [50]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [51]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

This script loads data in ```byregion.json``` and assigns it to ```jsondata```.

In [52]:
# Load JSON files and store the raw data in jsondata
with open("byregion.json", "r") as f:
    jsondata = json.load(f)

## Wrangle the data

Once we have the data we construct a ```DataFrame``` per region.

In [60]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def create_df(datalist, startdate, enddate):
    index=pd.date_range(startdate, enddate, freq='D')
    df = pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])
    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['cases', 'hospital', 'deaths']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(df.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                df.loc[date, column]=value
            
    # fill in any remaining "holes" due to missing dates
    df.fillna(0.0, inplace=True)
    return df

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dictionary 
    of DataFrames, one per region/area. """
    # get the list of data results
    datalist=jsondata['data']
    # get and sort by dates
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    # set of all the areas returned by the API/in the file
    areas = set(i['area'] for i in datalist)
    df_areas = {}
    for area in areas:
        datalist_area = [i for i in datalist if i['area'] == area]
        # create DF for area
        df_areas[area] = create_df(datalist_area, startdate, enddate)
    return df_areas

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API
dfs=wrangle_data(jsondata) # dfs is the dict of area & df

## Download current data

We create a "download" button to refresh the data from the file with live data from the API. If the call fails there is error handling to let the user know the call has failed.

In [54]:
# API access function for region data
def access_api():
    """ Accesses the PHE API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    filters = [
        'areaType=region'
    ]
    structure = {
        "area": "areaName",
        "date": "date",
        "cases": "newCasesBySpecimenDateRollingRate",
        "hospital": "newAdmissionsRollingRate",
        "deaths": "newDailyNsoDeathsByDeathDate" 
    }
    try:
        api = Cov19API(filters=filters, structure=structure)
        return api.get_json()
    except Exception as e:
        # returns 1 if there is an error
        return 1

In [55]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter.
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get API data
    apidata=access_api()
    if apidata == 1:
        # display error if there's a problem
        apibutton.disabled=True
        apibutton.description = 'Unavailable'
        apibutton.button_style = 'warning'
        return
    # wrangle the data and overwrite the dataframe for plotting
    global dfs
    dfs=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction
    refresh_graph()
    # display check & disable btn
    apibutton.icon="check"
    apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Download',
    disabled=False,
    button_style='info',
    tooltip="Download API data",
    icon='download'
)

apibutton.on_click(api_button_callback) # on_click callback

display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Download', icon='download', style=ButtonStyle(), tooltip='Download AP…

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [61]:
def plot_region(area):
    """ Our sample graph plotting function """
    dfs[area].plot()
    plt.show() # important! update won't work properly without this

# region selector widget
region_select=wdg.Dropdown(
    options=dfs.keys(),
    value=list(dfs.keys())[0],
    description='Region: ',
    disabled=False,
)

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick. """
    current=region_select.value
    if current==list(dfs.keys())[0]:
        other=list(dfs.keys())[1]
    else:
        other=list(dfs.keys())[0]
    region_select.value=other # forces the redraw
    region_select.value=current # now we can change it back
    
# connect the plotting function and the widget - the keys are syced to the DFs
# so this will always work
graph=wdg.interactive_output(plot_region, {'area': region_select})

# actually displays the graph
display(region_select, graph)

Dropdown(description='Region: ', options=('East of England', 'East Midlands', 'Yorkshire and The Humber', 'Nor…

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash). Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).